## Gradient Boosting with LightGBM (60/20/20 split)

Train a LightGBM classifier on the churn dataset using a stratified 60% train / 20% validation / 20% test split, optimising ROC-AUC with early stopping.

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, average_precision_score
)

from lightgbm import LGBMClassifier

import matplotlib.pyplot as plt
import seaborn as sns

import joblib

RANDOM_STATE = 42


### Step 1: Load Dataset

In [ ]:
DATA_PATH = Path('churn_clean.csv')
df = pd.read_csv(DATA_PATH)
df.head()


### Step 2: Prepare Features and Target

In [ ]:
X = df.drop('Exited', axis=1)
y = df['Exited']

cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(exclude='object').columns.tolist()
cat_cols, num_cols


### Step 3: Train/Validation/Test Split (60/20/20)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.4,
    stratify=y,
    random_state=RANDOM_STATE
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=RANDOM_STATE
)

print('Train:', X_train.shape, 'Target ratio:', y_train.mean().round(3))
print('Validation:', X_val.shape, 'Target ratio:', y_val.mean().round(3))
print('Test:', X_test.shape, 'Target ratio:', y_test.mean().round(3))


### Step 4: Build Preprocessing Pipeline

In [ ]:
try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

preprocess = ColumnTransformer([
    ('categorical', ohe, cat_cols),
    ('numeric', StandardScaler(), num_cols)
])

preprocess.fit(X_train)

X_train_prepared = preprocess.transform(X_train)
X_val_prepared = preprocess.transform(X_val)
X_test_prepared = preprocess.transform(X_test)

feature_names = preprocess.get_feature_names_out()
X_train_prepared.shape


### Step 5: Train LightGBM with Early Stopping

In [ ]:
lgbm_clf = LGBMClassifier(
    objective='binary',
    metric='auc',
    boosting_type='gbdt',
    num_leaves=63,
    max_depth=-1,
    learning_rate=0.05,
    n_estimators=2000,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
lgbm_clf.fit(
    X_train_prepared, y_train,
    eval_set=[(X_val_prepared, y_val)],
    eval_metric='auc',
    early_stopping_rounds=100,
    verbose=100
)


### Step 6: Evaluate Performance

In [ ]:
y_val_pred = lgbm_clf.predict(X_val_prepared)
y_val_proba = lgbm_clf.predict_proba(X_val_prepared)[:, 1]
y_test_pred = lgbm_clf.predict(X_test_prepared)
y_test_proba = lgbm_clf.predict_proba(X_test_prepared)[:, 1]


In [ ]:
print('Validation ROC-AUC:', round(roc_auc_score(y_val, y_val_proba), 3))
print(classification_report(y_val, y_val_pred))
print('Test ROC-AUC:', round(roc_auc_score(y_test, y_test_proba), 3))
print(classification_report(y_test, y_test_pred))


In [ ]:
val_cm = confusion_matrix(y_val, y_val_pred)
test_cm = confusion_matrix(y_test, y_test_pred)
val_cm, test_cm


### Step 7: Visual Diagnostics

In [ ]:
FIG_DIR = Path('figures')
FIG_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
val_fpr, val_tpr, _ = roc_curve(y_val, y_val_proba)
test_fpr, test_tpr, _ = roc_curve(y_test, y_test_proba)
val_auc = roc_auc_score(y_val, y_val_proba)
test_auc = roc_auc_score(y_test, y_test_proba)

val_prec, val_rec, _ = precision_recall_curve(y_val, y_val_proba)
test_prec, test_rec, _ = precision_recall_curve(y_test, y_test_proba)
val_ap = average_precision_score(y_val, y_val_proba)
test_ap = average_precision_score(y_test, y_test_proba)


In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(val_fpr, val_tpr, label=f'Validation AUC = {val_auc:.3f}')
plt.plot(test_fpr, test_tpr, label=f'Test AUC = {test_auc:.3f}', linestyle='--')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve — LightGBM Baseline')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig(FIG_DIR / 'roc_curve_lgbm_baseline.png', dpi=200)
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(val_rec, val_prec, label=f'Validation AP = {val_ap:.3f}')
plt.plot(test_rec, test_prec, label=f'Test AP = {test_ap:.3f}', linestyle='--')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision–Recall Curve — LightGBM Baseline')
plt.legend(loc='lower left')
plt.tight_layout()
plt.savefig(FIG_DIR / 'pr_curve_lgbm_baseline.png', dpi=200)
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 4))
sns.heatmap(val_cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0])
axes[0].set_title('Validation Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
sns.heatmap(test_cm, annot=True, fmt='d', cmap='Greens', cbar=False, ax=axes[1])
axes[1].set_title('Test Confusion Matrix')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
plt.tight_layout()
plt.savefig(FIG_DIR / 'confusion_matrices_lgbm_baseline.png', dpi=200)
plt.show()


### Step 8: Feature Importance

In [ ]:
importances = pd.Series(lgbm_clf.feature_importances_, index=feature_names)
top_features = importances.sort_values(ascending=False).head(20)
top_features


In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x=top_features.values, y=top_features.index, color='tab:green')
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.title('Top 20 Features — LightGBM Baseline')
plt.tight_layout()
plt.show()


### Step 9: Persist Model and Preprocessor

In [ ]:
MODEL_DIR = Path('models')
MODEL_DIR.mkdir(parents=True, exist_ok=True)
MODEL_PATH = MODEL_DIR / 'lgbm_baseline.joblib'
joblib.dump({'model': lgbm_clf, 'preprocess': preprocess}, MODEL_PATH)
MODEL_PATH


### Step 10: Notes

- Early stopping monitors validation AUC and prevents overfitting.
- Compare ROC-AUC/PR-AUC against other baselines to gauge lift.
- Reload the saved joblib file to score new records with identical preprocessing.